In [1]:
#IMPORT NECESSARY PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
import pymysql.cursors
import datetime
import matplotlib.gridspec as grd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import datetime
from time import mktime

#IMPORT NECESSARY PACKAGES
import CO2_functions
from CO2_functions import *

In [129]:
import importlib
importlib.reload(CO2_functions)
from CO2_functions import *

In [66]:
date1,date2 = get_date_range()
data = get_sql_data("Aug2019_LI_8100_Vent",\
              "Aug2019_Multiplexer","Aug2019_Vent_Anem_Temp",\
              "Aug2019_Picarro",date1,date2,'all','split')
data = drift_correct(data)

Enter Start Date YYYY-mm-DD:  2019-10-01
Enter End Date YYYY-mm-DD:  2019-10-01
